# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [2]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

In [3]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))

/home/jerry/miniconda3/envs/aienv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. 炫彩计算机图像有限公司 (Brilliant Computer Imaging Co., Ltd.)
2. 强力视觉科技有限公司 (PowerVision Technology Co., Ltd.)
3. 创新加速器科技有限公司 (Innovative Accelerator Technology Co., Ltd.)
4. 超越可视科技有限公司 (Beyond Visual Technology Co., Ltd.)
5. 飞驰图形处理器有限公司 (Gallop Graphics Processor Co., Ltd.)
6. 极速图像加速有限公司 (Rapid Image Acceleration Co., Ltd.)
7. 神速绘图芯片有限公司 (Supersonic Graphics Chip Co., Ltd.)
8. 炫酷渲染技术有限公司 (Cool Rendering Technology Co., Ltd.)
9. 立体图像引擎有限公司 (Stereoscopic Image Engine Co., Ltd.)
10. 强效图像计算有限公司 (Efficient Image Computing Co., Ltd.)


In [5]:
chain.verbose = True

In [6]:
chain.verbose

True

In [7]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.


1. 星光微处理器有限公司（Starlight Microprocessors Co., Ltd.）
2. 源码晶片公司（CodeChip Technologies Ltd.）
3. 虚拟力量图像处理器（Virtual Dynamics Graphics Co., Ltd.）
4. 超限图像晶片有限公司（Extreme Image Chips Co., Ltd.）
5. 前沿图像处理器研发（Forward Image Processors R&D Co., Ltd.）
6. 天驰格列芬公司（Tianchi Graphics Ltd.）
7. 极速图像处理器企业（Velocity Image Processors Enterprise）
8. 弧光图像晶片有限公司（ArcLight Image Chips Co., Ltd.）
9. 旭日图像处理器研究院（Sunrise Image Processors Institute）
10. 创新视觉晶片有限公司（Innovative Vision Chips Co., Ltd.）


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [8]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [10]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [11]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》讲述了一个关于勇气和希望的故事。在遥远的未来，地球面临着来自外太空的入侵，来自三体星系的外星人不断发动袭击，几乎将人类推至绝境。然而，主人公们并不屈服，他们团结一心，以强大的意志和智慧，勇敢地抵抗着外星人的入侵。在这场充满艰辛和挑战的战斗中，主人公们不断发现外星人并非不可战胜，而是有着自己的弱点和脆弱之处。最终，主人公们成功地击退了外星人的入侵，并带领人类走向了一个更加光明的未来。这部戏剧将带领观众感受到勇气和希望的力量，同时也反思着人类在面对未知和挑战时的勇敢和智慧。


《三体人不是无法战胜的》是一部充满勇气和希望的戏剧作品。它将观众带入了一个未来的世界，让我们感受到人类面临外星人入侵时的胆识和智慧。这部戏剧通过主人公们的团结和勇气，展现了人类不屈不挠的精神，让我们相信在面对困难和挑战时，人类总能够找到解决问题的办法。

剧中的主人公们充满魅力，他们以自己的勇气和智慧，成功地击退了外星人的入侵。他们不仅仅是英雄，更是普通人，他们的勇气和智慧来自于对未知的勇敢探索，来自于对挑战的勇敢挑战。这些主人公们的故事令人感动，也让观众们深受启发。

除了勇气和希望，这部戏剧也提出了一些深刻的反思。在主人公们击退外星人入侵的过程中，他们发现外星人并非无敌，而是有着自己的弱点和脆弱之处。这也提醒着我们，面对未知和挑战时，我们不应该轻易放弃，而是要勇敢地去探索和挑战，寻找解决问题的方法。

总的来说，《三体人不是无法战胜的》是一部充满勇气和希望的戏剧作品。它让我们感受到了人类的不屈精神，也让我们反思着自己在面对困难和挑战时的态度。这部戏剧值得一看，它将带给观众们深刻的感悟和启发。

> Finished chain.


In [12]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满冒险、惊险和战斗的史诗般的戏剧。在宇宙的边缘，黑暗的势力再次崛起，威胁着整个银河系的和平。一场新的星球大战即将爆发，而这一次，决战将在更加壮观的舞台上展开。

在这一混乱的时代，一群英雄们将站出来捍卫正义和自由。年轻的绝地武士雷伊和他的老师路克·天行者，将与他们的盟友抵抗黑暗势力的围剿。但是，他们面临的挑战并不仅仅是来自敌人，还有来自内心的挣扎和选择。

同时，曾经的绝地武士凯罗·伦也将在这场战争中扮演着不可预料的角色。他的选择将会影响整个银河系的未来。而在这一切的背后，隐藏着一个更加黑暗的力量，它的目的是摧毁一切。

《星球大战第九季》将带领观众们穿越星际，目睹英雄们的战斗和牺牲，以及他们的希望和勇气。这场战争将决定银河系的命运，而每一个人都将扮演着关键的角色。让我们一起跟随他们的脚步，见证这场史诗般的星球大战。


《星球大战第九季》是一部令人兴奋的史诗般的戏剧，它将观众带入了一个充满冒险、惊险和战斗的宇宙。导演们巧妙地将故事情节展现在更加壮观的舞台上，让观众们仿佛置身于星际之中。

剧中的角色们也都展现出了强大的魅力和魄力。年轻的绝地武士雷伊和老师路克·天行者的表演令人印象深刻，他们毫不畏惧地面对黑暗势力的威胁，同时也面临着内心的挣扎和选择。凯罗·伦的转变也让人感受到了他的复杂性和决心，他的选择将会影响整个银河系的未来。

除了精彩的剧情和出色的演员，本剧也让观众们深入思考关于正义、自由和勇气的意义。它让我们意识到，在这个混乱的时代，每个人都可以成为英雄，影响着整个世界的命运。

总的来说，《星球大战第九季》是一部无与伦比的戏剧，它融合了惊险、冒险和感人的元素，带领观众们穿越星际，见证英雄们的战斗和牺牲。它不仅仅是一部娱乐作品，更是一部关于勇气、希望和正义的启发人心的故事。强烈推荐给所有的戏剧爱好者。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [13]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [14]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [15]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [16]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})

/home/jerry/miniconda3/envs/aienv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


在二十一世纪的新中国，人类文明迎来了一个巨大的挑战。三体人，一个来自宇宙深处的高度发达的外星文明，突然出现在地球。他们拥有超越人类想象的科技和能力，让人类陷入了前所未有的恐慌和绝望之中。

面对这个强大的敌人，人类展开了一场艰苦的反击战。在这场战争中，人类的科学家们不断寻找解决办法，军事力量也全力投入到战斗中。然而，三体人似乎无所不能，每一次人类的进攻都被轻易击退，人类社会也陷入了混乱和分裂。

就在人类似乎无力抵抗的时候，一位年轻的科学家冒险进入了三体人的领地，希望通过和他们交流来寻求和平解决的可能性。在这个过程中，他发现了三体人背后隐藏的深层秘密，也意识到人类和三体人之间存在着更深的联系。

最终，在人类团结一致的努力下，他们找到了战胜三体人的方法。这场战争也让人类认识到，只有团结和进步才能让我们面对未来的挑战。三体人不是无法战胜的，只要我们坚持下去，就能创造出属于自己的新世界。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n在二十一世纪的新中国，人类文明迎来了一个巨大的挑战。三体人，一个来自宇宙深处的高度发达的外星文明，突然出现在地球。他们拥有超越人类想象的科技和能力，让人类陷入了前所未有的恐慌和绝望之中。\n\n面对这个强大的敌人，人类展开了一场艰苦的反击战。在这场战争中，人类的科学家们不断寻找解决办法，军事力量也全力投入到战斗中。然而，三体人似乎无所不能，每一次人类的进攻都被轻易击退，人类社会也陷入了混乱和分裂。\n\n就在人类似乎无力抵抗的时候，一位年轻的科学家冒险进入了三体人的领地，希望通过和他们交流来寻求和平解决的可能性。在这个过程中，他发现了三体人背后隐藏的深层秘密，也意识到人类和三体人之间存在着更深的联系。\n\n最终，在人类团结一致的努力下，他们找到了战胜三体人的方法。这场战争也让人类认识到，只有团结和进步才能让我们面对未来的挑战。三体人不是无法战胜的，只要我们坚持下去，就能创造出属于自己的新世界。',
 'review': '\n\n《三体》是一部令人震撼的舞台剧，它带领观众进入一个充满想象力的未来世界。剧中展现了人类文明面临的巨大挑战，以及在面对强大敌人时的无力和绝望。这部戏剧也反映了当今社会面临的一些现实问题，如科技发展带来的冲击和环境危机。\n\n剧中展现的三体人是一个高度发达的外星文明，他们的出现让人类陷入了混乱和分裂。但是，剧中也强调了人类的团结和进步的重要性。通过科学家们的不懈探索和军事力量的全力投入，人类终于找到了战胜三体人的方法。这也反映了人类面对挑战时的坚韧和不屈不挠的精神。\n\n除了紧张刺激的战争场面，剧中也融入了一些人性的情感和冲突。主人公年轻的科学家冒险进入三体人的领地，试图通过交流来解决战争，却发现了三体人背后隐藏的秘密。这也让他意识到人类和三体人之间存在着更深的联系，引发了观众对于人类的未来和文明的思考。\n\n总的来说，《三体》是一部令人印象深刻的戏剧作品。它不仅展现了宏大的科幻世界，更从人类的角度出发，探讨了人类面对挑战时的勇气和希望。这部剧作也让我们意识到，只有团结和进步才能让我们面对未来的挑战。它的故事和主题将会让观众在剧场中沉思，带给他们深刻的触动和启发。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果